## Parse Messenger inbox

In [ ]:
from datetime import datetime
import glob
import json

In [ ]:
chats = glob.glob("facebook-jasoncbenn/messages/inbox/*/*.json")

In [ ]:
from datetime import timedelta
import unidecode
from collections import defaultdict

MY_NAME = 'Jason Benn'

interactions = defaultdict(lambda: {'interaction_days': set()})

def parse_datetime(dt: int) -> datetime:
    return datetime.fromtimestamp(dt / 1e3)

for chat_json_path in chats:
    with open(chat_json_path, encoding='raw_unicode_escape') as f:
        chat = json.loads(f.read().encode('raw_unicode_escape').decode())

    others = [x['name'] for x in chat['participants'] if x['name'] != MY_NAME]

    my_messages = [x for x in chat['messages'] if x['sender_name'] == MY_NAME]

    if not len(my_messages):
        continue 

    last_message_time = None

    is_groupchat = len(others) > 1

    if is_groupchat:
        # Creative technique: anytime I message the group, I get "credit" for all messages within 2 days
        # Goal is to account for groups where I'm active + others are active - those feel like touches
        # But many other groups are too big, too sparse, not mine, etc. Those don't count.
        for message in reversed(chat['messages']):
            msg_time = parse_datetime(message['timestamp_ms'])
            msg_name = unidecode.unidecode(message['sender_name'])
            if msg_name == MY_NAME:
                last_message_time = msg_time
                continue

            if last_message_time and last_message_time + timedelta(days=2) > msg_time:
                interactions[msg_name]['interaction_days'].add(msg_time.date())

    else:
        # If this is 1:1, just count every unique day of any message.
        for message in chat['messages']:
            msg_time = parse_datetime(message['timestamp_ms'])
            msg_name = unidecode.unidecode(message['sender_name'])
            if msg_name == MY_NAME:
                continue
            interactions[msg_name]['interaction_days'].add(msg_time.date())

In [ ]:
compressed_interactions = {}
for name, touches in interactions.items():
    compressed_interactions[name] = {'last_interaction': max(touches['interaction_days']), 'num_interaction_days': len(touches['interaction_days'])}

sorted(compressed_interactions.items(), key=lambda x: -x[1]['num_interaction_days'])

## Send to Notion